In [1]:
import pandas as pd
import os
import numpy as np
import requests
from io import BytesIO
from zipfile import ZipFile

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
DATA_SOURCE_URL = 'https://caaspp-elpac.cde.ca.gov/caaspp/researchfiles/sb_ca2019_all_csv_v4.zip'
TESTS_URL = 'https://caaspp-elpac.cde.ca.gov/caaspp/researchfiles/tests.zip'
SUBGROUPS_URL = 'https://caaspp-elpac.cde.ca.gov/caaspp/researchfiles/subgroups.zip'

In [4]:
METRICS_FIELDS = [
    'Mean Scale Score',
    'Percentage Standard Exceeded',
    'Percentage Standard Met',
    'Percentage Standard Met and Above',
    'Percentage Standard Nearly Met',
    'Percentage Standard Not Met',
]
INDICATOR = 'Percentage Standard Met and Above'
SFUSD = 68478
SF_COUNTY_NAME = 'San Francisco'
GRADES = [3, 4, 5]

In [5]:
r = requests.get(DATA_SOURCE_URL).content
zipfile = ZipFile(BytesIO(r))
for filename in zipfile.namelist():
    if 'entities' in filename:
        entities = pd.read_csv(
            zipfile.open(filename),
            encoding='iso-8859-1')
    else:
        results = pd.read_csv(
            zipfile.open(filename),
            encoding='iso-8859-1',
            dtype={field: np.float64 for field in METRICS_FIELDS},
            na_values=['*']
        )

In [6]:
def get_dataframe_from_zipfile_url(url):
    r = requests.get(url).content
    zipfile = ZipFile(BytesIO(r))
    return pd.read_csv(
        zipfile.open(zipfile.namelist()[0]),
        encoding='iso-8859-1',
    )
tests = get_dataframe_from_zipfile_url(TESTS_URL).set_index('Test ID')
results.drop([colname for colname in tests.columns if colname in results.columns], axis=1, inplace=True)
subgroups = get_dataframe_from_zipfile_url(SUBGROUPS_URL).set_index('Demographic ID')
subgroups['Subgroup Label'] = subgroups["Student Group"] + ': ' + subgroups["Demographic Name"]
results.drop([colname for colname in subgroups.columns if colname in results.columns], axis=1, inplace=True)
results = results.join(tests, on='Test Id')
results = results.join(subgroups, on='Subgroup ID')
sfusd_entities = entities[entities['District Name'] == 'San Francisco Unified']
sfusd = results[results['District Code'] == SFUSD]

In [7]:
sfusd = sfusd.join(sfusd_entities.set_index('School Code')[['School Name', 'County Name', 'District Name']], on='School Code')

In [9]:
for grade in GRADES:
    for subgroup_id, subgroup in subgroups.to_dict(orient='index').items():
        if subgroup_id not in [204]:
            continue
        tmp = sfusd[
            (~sfusd[INDICATOR].isna()) &
            (sfusd['Grade'] == grade) & 
            (sfusd['Subgroup ID'] == subgroup_id)
        ][['School Name', 'Students Tested', 'Test Id', INDICATOR]]
        out = pd.pivot_table(
            tmp,
            values=INDICATOR, index=['School Name', 'Students Tested'],
            columns=['Test Id'], aggfunc=np.sum
        )
        out.index.names = list(map(lambda name: tests.get(name, name), out.index.names))
        out.index.names = list(map(lambda name: name.replace('Elementary', ''), out.index.names))
        out.rename(columns=tests['Test Name'].to_dict(), inplace=True)
        print(f'Grade {grade}')
        print(list(subgroup.values())[-1])
        print(INDICATOR)
        try:
            for colname in list(out.columns):
                display(out[[colname]].sort_values(colname, ascending=False).dropna())
        except IndexError:
            print('(no data)')

Grade 3
Ethnicity for Economically Disadvantaged: Hispanic or Latino
Percentage Standard Met and Above


,Test Id,SB - English Language Arts/Literacy
School Name,Students Tested,
Hillcrest Elementary,26.0,65.38
Muir (John) Elementary,16.0,37.50
Moscone (George R.) Elementary,32.0,37.50
Spring Valley Elementary,19.0,36.84
Rooftop Elementary,12.0,33.33
Serra (Junipero) Elementary,23.0,30.43
Alvarado Elementary,20.0,30.00
Taylor (Edward R.) Elementary,30.0,30.00
Webster (Daniel) Elementary,11.0,27.27


,Test Id,SB - Mathematics
School Name,Students Tested,
Hillcrest Elementary,26.0,96.15
Moscone (George R.) Elementary,32.0,50.00
Taylor (Edward R.) Elementary,31.0,48.39
Spring Valley Elementary,19.0,47.37
Muir (John) Elementary,16.0,43.75
Serra (Junipero) Elementary,23.0,43.48
Webster (Daniel) Elementary,11.0,36.36
Bryant Elementary,18.0,33.33
Monroe Elementary,31.0,25.81


Grade 4
Ethnicity for Economically Disadvantaged: Hispanic or Latino
Percentage Standard Met and Above


,Test Id,SB - English Language Arts/Literacy
School Name,Students Tested,
Redding Elementary,11.0,54.55
Alvarado Elementary,19.0,42.11
Muir (John) Elementary,14.0,35.71
Taylor (Edward R.) Elementary,24.0,33.33
Marshall Elementary,18.0,33.33
Chavez (Cesar) Elementary,46.0,32.61
Monroe Elementary,37.0,32.43
Guadalupe Elementary,31.0,29.03
Glen Park Elementary,22.0,27.27


,Test Id,SB - Mathematics
School Name,Students Tested,
Muir (John) Elementary,15.0,46.67
Redding Elementary,11.0,45.45
Alvarado Elementary,19.0,36.84
Moscone (George R.) Elementary,33.0,36.36
Webster (Daniel) Elementary,15.0,33.33
Taylor (Edward R.) Elementary,25.0,32.00
Chavez (Cesar) Elementary,46.0,28.26
Bryant Elementary,29.0,27.59
Glen Park Elementary,22.0,27.27


Grade 5
Ethnicity for Economically Disadvantaged: Hispanic or Latino
Percentage Standard Met and Above


,Test Id,SB - English Language Arts/Literacy
School Name,Students Tested,
Muir (John) Elementary,18.0,61.11
Alvarado Elementary,16.0,43.75
Serra (Junipero) Elementary,21.0,42.86
Hillcrest Elementary,24.0,33.33
Flynn (Leonard R.) Elementary,31.0,32.26
Monroe Elementary,22.0,31.82
Marshall Elementary,26.0,30.77
Guadalupe Elementary,24.0,29.17
Tenderloin Community,11.0,27.27


,Test Id,SB - Mathematics
School Name,Students Tested,
Muir (John) Elementary,19.0,78.95
Alvarado Elementary,16.0,31.25
Carmichael (Bessie)/FEC,12.0,25.00
Webster (Daniel) Elementary,15.0,20.00
San Francisco Community Alternative,11.0,18.18
Tenderloin Community,11.0,18.18
Glen Park Elementary,22.0,18.18
Hillcrest Elementary,23.0,17.39
Guadalupe Elementary,24.0,16.67
